In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv('Google_Stock_Price_Train.csv')
data.head()

,Date,Open,High,Low,Close,Volume
0,1/3/2012,325.25,332.83,324.97,663.59,"7,380,500"
1,1/4/2012,331.27,333.87,329.08,666.45,"5,749,400"
2,1/5/2012,329.83,330.75,326.89,657.21,"6,590,300"
3,1/6/2012,328.34,328.77,323.68,648.24,"5,405,900"
4,1/9/2012,322.04,322.29,309.46,620.76,"11,688,800"


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1258 entries, 0 to 1257
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    1258 non-null   object 
 1   Open    1258 non-null   float64
 2   High    1258 non-null   float64
 3   Low     1258 non-null   float64
 4   Close   1258 non-null   object 
 5   Volume  1258 non-null   object 
dtypes: float64(3), object(3)
memory usage: 59.1+ KB


In [4]:
data_set = data[['Open']].values

In [5]:
train = data_set[:len(data_set)-50]
test = data_set[len(train):]
train.reshape(train.shape[0],1)
train.shape

(1208, 1)

In [6]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
train_scaler = scaler.fit_transform(train)

In [7]:
X_train = []
Y_train = []
timesteps = 50
for i in range(timesteps, len(train_scaler)):
    X_train.append(train_scaler[i-timesteps:i,0])
    Y_train.append(train_scaler[i,0])
X_train, Y_train = np.array(X_train), np.array(Y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))

In [11]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Dropout

In [13]:
regressor = Sequential()
regressor.add(SimpleRNN(units=50, activation = 'tanh', return_sequences=True, input_shape=(X_train.shape[1],1)))
regressor.add(Dropout(0.2))
regressor.add(SimpleRNN(units=50, activation = 'tanh', return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(SimpleRNN(units=50, activation = 'tanh', return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(SimpleRNN(units=50))
regressor.add(Dropout(0.2))
regressor.add(Dense(units=1))

In [14]:
regressor.compile(optimizer='adam', loss='mean_squared_error')

In [15]:
regressor.fit(X_train, Y_train, epochs=20, batch_size=16)

Epoch 1/20

73/73 [==============================] - 10s 34ms/step - loss: 0.4234
Epoch 2/20
73/73 [==============================] - 2s 34ms/step - loss: 0.2622
Epoch 3/20
73/73 [==============================] - 2s 34ms/step - loss: 0.2007
Epoch 4/20
73/73 [==============================] - 2s 34ms/step - loss: 0.1589
Epoch 5/20
73/73 [==============================] - 2s 34ms/step - loss: 0.1422
Epoch 6/20
73/73 [==============================] - 2s 34ms/step - loss: 0.0883
Epoch 7/20
73/73 [==============================] - 3s 35ms/step - loss: 0.0719
Epoch 8/20
73/73 [==============================] - 3s 35ms/step - loss: 0.0625
Epoch 9/20
73/73 [==============================] - 3s 35ms/step - loss: 0.0463
Epoch 10/20
73/73 [==============================] - 3s 35ms/step - loss: 0.0419
Epoch 11/20
73/73 [==============================] - 3s 35ms/step - loss: 0.0366
Epoch 12/20
73/73 [==============================] - 2s 34ms/step - loss: 0.0318
Epoch 13/20
73/73 [================

In [16]:
inputs = data_set[len(data_set)-len(test)-timesteps:]
inputs = scaler.transform(inputs)

In [17]:
X_test = []
for i in range(timesteps, inputs.shape[0]):
    X_test.append(inputs[i-timesteps:i,0])
X_test_RNN = np.array(X_test)
X_test_RNN = np.reshape(X_test_RNN, (X_test_RNN.shape[0], X_test_RNN.shape[1],1))

In [18]:
predicted_stock = regressor.predict(X_test_RNN)
predicted_stock = scaler.inverse_transform(predicted_stock)

2/2 [==============================] - 1s 18ms/step


In [20]:
from sklearn import metrics
mae = metrics.mean_absolute_error(test, predicted_stock)
print('Mean Absolute Error: ',mae)
mse = metrics.mean_squared_error(test, predicted_stock)
rmse = np.sqrt(mse)
print('Root Mean Squared Error: ',rmse)

mape = metrics.mean_absolute_percentage_error(test, predicted_stock)
accuracy = 100 - mape*100
print('Accuracy: ', accuracy)

Mean Absolute Error:  58.597601074218744
Root Mean Squared Error:  60.05422772618663
Accuracy:  92.45030726347728
